anxienet_model.ipynb：AnxieNet模型的训练、调优和评估过程

此笔记本包含AnxieNet模型的实现，使用Transformer和强化学习进行焦虑检测任务的训练和评估。它提供了完整的模型训练和评估代码。

In [ ]:
# anxienet_model.ipynb

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np


In [ ]:
# 1. 定义AnxieNet模型（Transformer + 强化学习优化）
class AnxieNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads=8):
        super(AnxieNet, self).__init__()
        
        # Transformer部分
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads), 
            num_layers=6
        )
        
        # 输出层
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.fc(x.mean(dim=1))  # 使用平均池化
        return x

In [ ]:
# 2. 加载数据（假设已经提取并标准化了特征）
X = np.random.randn(100, 256)  # 随机特征数据
y = np.random.randint(0, 2, 100)  # 随机标签，假设为二分类任务

# 转换为PyTorch张量
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)


In [ ]:
# 3. 数据分割：训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2)


In [ ]:
# 4. 实例化模型
model = AnxieNet(input_dim=256, hidden_dim=128, output_dim=2)


In [ ]:
# 5. 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 6. 训练过程
def train(model, X_train, y_train, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

In [ ]:
# 7. 评估过程
def evaluate(model, X_val, y_val):
    model.eval()
    with torch.no_grad():
        output = model(X_val)
        _, predicted = torch.max(output, 1)
        accuracy = accuracy_score(y_val, predicted)
        print(f"Accuracy: {accuracy:.4f}")


In [ ]:
# 8. 训练和评估模型
train(model, X_train, y_train, criterion, optimizer)
evaluate(model, X_val, y_val)